# Question 2

## 1. Index
1. Index
2. Problem definition
3. Solution strategy
4. Design choies
5. Implementation approach
6. Implementation
7. Discussion

## 2. Problem definition

- Reshaped holds six discovery meetings weekly and prioritizes gaining a deep understanding of potential clients beforehand.  
- A key part of this preparation involves reviewing annual reports,  
  - A process that is often time-consuming due to their length and inconsistent formatting.  
- To streamline this process, we aim to leverage a **Large Language Model (LLM)**.  
- These annual reports contain tables:  
  - These tables need to be **accurately extracted and formatted** using an LLM.

## 3. Solution Strategy

- Python libraries exist that can **extract tables from PDFs**,  
  - However, they may not preserve the original formatting.

- Since table formats can vary between files, hardcoding the parsing process is not ideal.

- **A flexible method** for parsing tables can be built using LLMs:  
  1. Extract the table from the PDF using a designated library.  
  2. Instruct the LLM to infer the format from the extracted text.  
  3. Return an easy-to-manipulate text object, such as a CSV.


![title](docs/img/flowchart_table_extraction.jpg)

## 4. Design choices

Every step in the process described above involves making key choices. I will go over the **main decisions and highlight alternatives**.

**PS:**  
- There's no strict need to use an LLM to format the table (as I will demonstrate in section 6).  
  - The `Camelot` library **can extract all tables from the document** in their original format.  
  - This approach is preferred due to its simplicity and its ability to process all tables in a document.  
  - However, to stay aligned with the task, I will demonstrate using LLMs anyway.


### 4.1 Text extraction

- Many libraries can extract text from PDF files (e.g., `PyPDF2`, `Tika`, LangChain’s `PyPDFLoader`).

- There are also libraries specifically **dedicated to extracting tables** from PDFs, such as:  
  - `Camelot`  
  - `Tabula`  
  - `PDFPlumber`

- Most libraries are not capable of reading the full contents of a table:  
  - For example, `PDFPlumber` considers only grey-highlighted rows.

- Worked with `PyMuPDF` to read the table as text and format it later.

<br></br>

- Alternatively, one could read every page of the PDF as an image and process it using **OCR**
- OCR-based formatting is flexible because it allows extraction of all tables without hardcoding their location:  
  - However, OCR is more expensive.  
  - `GPT-3.5 Turbo` is not capable of OCR.

- Opt for "regular" text formatting.


### 4.2 LLM Formatting

- Obtain the contents of the table in text format:  
  - For flexibility, use an **LLM to handle formatting**.

- Provide LLM instructions on how to process the extracted table text:  
  - Format of input  
  - Processing steps  
  - Desired output

- To ensure we stick to facts, set `temperature = 0`.


![title](docs/img/q2_system_prompt.PNG)

### 4.3 DataFrame conversion
- LLM returns semicolon separated text
    - In order to preserve comma's in table

- Simple `pd.read_csv()` call sufficient to return a Pandas DataFrame

### 4.4 design overview
- Text loading = `PyMuPDF`
- Text processing = `GPT 3.5 Turbo`, `temperature = 0`
- Dataframe = `pd.read_csv()`

## 5 Implementation approach
- Developed implementation of design above by:
    - Iteratively exploring design choices using ChatGPT
    - Experimenting with design choices using code snippets implemented by Cursor
    - Implementing folder structure, `utils.py`, and `README` with Cursor

## 6 Implementation

### 6.1 Setting up the environment

In [ ]:
#Import libraries
import importlib
import os
import utils
from dotenv import load_dotenv

# Reload utils to ensure changes in functions carry over
importlib.reload(utils)

# Load environment variables
load_dotenv()

# Specify document location
PDF_PATH = r"Microsoft_2023_Trimmed.pdf"

### 6.2 Extract table text from PDF

In [ ]:
# Extract table text from PDF
table_text = utils.extract_last_page_text(PDF_PATH)

### 6.3 Format text with LLM

In [ ]:
# Format text
df = utils.extract_tables_with_llm(table_text)

### 6.4 Inspect result

In [ ]:
print(df)

### 6.5 Extract table without LLM

In [ ]:
# # Extract tables
# pdf_tables = utils.extract_tables_with_camelot(pdf_path = PDF_PATH)

# # Print table on last page
# print(pdf_tables[-1])

## 7. Discussion